In [10]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class State(TypedDict):
    messages: Annotated[list[dict], add_messages]
    requirement: Annotated[list[tuple[str, str]], add_messages]
    response: str
    evaluation: str
    
    
graph_builder = StateGraph(State)

In [20]:
import os
from langchain.chat_models import init_chat_model

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("API_VERSION")

llm = init_chat_model(
    "azure_openai:gpt-4o",
    azure_deployment=os.environ["DEPLOYMENT_NAME"],
)

In [33]:
import sys
def agent1(state:State): #英訳する関数（後でノードとして定義し利用する）
    # query = state['query']
    messages = state['messages']
    
    # Prompt template(書き方2)
    prompt1 = ChatPromptTemplate([
      ('system', 'あなたは、Azure のアーキテクトです、ユーザの実行したいことを実現するために必要なヒアリング項目を1つだけ質問してください。なお、過去の会話履歴はこちらです。{messages}'),
      {"role": "user", "content": "ユーザーの要件を深掘りするための質問をしてください。ただし、質問は一つだけにしてください。分量は短く、簡潔にしてください。"},
    ])

    agent1 = (
        prompt1
        | llm
        | StrOutputParser()
    )
    
    response = agent1.invoke(messages)
    print(response)
    try:
        user_input = input("User: ")
    except:
        sys.exit(1)
    return {"requirement": [response, user_input], "messages": [{"role":"assistant","content":response},{"role": "user", "content": user_input}]}

In [34]:
def evaluator(state:State): #英訳する関数（後でノードとして定義し利用する）
    messages = state['messages']
    # Prompt template(書き方2)
    prompt = ChatPromptTemplate([
      ('system', 'あなたはAzureのアーキテクトです。Azure 環境構築時のヒアリング項目のリストが十分であれば yes、そうでなければ no と答えてください。'),
      ('user', """{messages}""")
    ])

    agent2 = (
        prompt
        | llm
        | StrOutputParser()
    )

    evaluation = agent2.invoke(messages).lower()
    print(evaluation)
    return {"evaluation": evaluation}

In [35]:
def loop_or_end(state):
    # quality_okがFalseなら"再度意図判定(classify)"へ、それ以外は"end"
    return "agent_1" if not state["evaluation"]=="yes" else "end"

In [36]:
workflow = StateGraph(State)

workflow.add_node("agent_1", agent1)
workflow.add_node("evaluator", evaluator)


workflow.add_edge(START, "agent_1")
workflow.add_edge("agent_1", "evaluator")
workflow.add_conditional_edges("evaluator", loop_or_end, {
    "agent_1": "agent_1",
    "end": END
})

graph_app = workflow.compile()

In [37]:
initial_state = State(query='Azure上にシステムを構築したい') #Agentに翻訳してほしい言葉を入れる

In [38]:
# streamを利用する場合
for chunk in graph_app.stream(initial_state):
    print(chunk)

# {'agent1': {'query': 'こんにちは', 'answer': 'Hello.'}}


そのシステムはどのような目的で使用しますか？
{'agent_1': {'requirement': ['そのシステムはどのような目的で使用しますか？', '簡単なアプリケーションのホスト'], 'messages': [{'role': 'assistant', 'content': 'そのシステムはどのような目的で使用しますか？'}, {'role': 'user', 'content': '簡単なアプリケーションのホスト'}]}}
no
{'evaluator': {'evaluation': 'no'}}
そのアプリケーションはどの程度のトラフィックを想定していますか？
{'agent_1': {'requirement': ['そのアプリケーションはどの程度のトラフィックを想定していますか？', '自分のみの検証用'], 'messages': [{'role': 'assistant', 'content': 'そのアプリケーションはどの程度のトラフィックを想定していますか？'}, {'role': 'user', 'content': '自分のみの検証用'}]}}
no
{'evaluator': {'evaluation': 'no'}}
アプリケーションの使用言語とフレームワークは何ですか？
{'agent_1': {'requirement': ['アプリケーションの使用言語とフレームワークは何ですか？', 'PythonでLangChain'], 'messages': [{'role': 'assistant', 'content': 'アプリケーションの使用言語とフレームワークは何ですか？'}, {'role': 'user', 'content': 'PythonでLangChain'}]}}
no
{'evaluator': {'evaluation': 'no'}}
アプリケーションに必要な計算リソースはどの程度ですか？
{'agent_1': {'requirement': ['アプリケーションに必要な計算リソースはどの程度ですか？', '動けばいい'], 'messages': [{'role': 'assistant', 'content': 'アプリケーションに必要な計算リソースはどの程度ですか？'}, {'role': 'u

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass